In [ ]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load the Postgres extension
duckdb.sql("INSTALL postgres_scanner;")
duckdb.sql("LOAD postgres_scanner;")

# credentials
pg_conn_str = "dbname='mimic-iii' user='sangeethgeorge' password='12345' host='localhost' port='5432'"

#SQL query to connect to DB
labs48_query = """
SELECT *
FROM postgres_scan('dbname=mimic-iii user=sangeethgeorge password=12345 host=localhost port=5432', 
                   'public', 'all_labs_48h')
"""
vitals48_query = """
SELECT *
FROM postgres_scan('dbname=mimic-iii user=sangeethgeorge password=12345 host=localhost port=5432', 
                   'public', 'all_vitals_48h')
"""

onc_cohort = """
SELECT *
FROM postgres_scan('dbname=mimic-iii user=sangeethgeorge password=12345 host=localhost port=5432', 
                   'public', 'oncology_icu_base')
"""

labs48_df= duckdb.sql(labs48_query).df()
vitals48_df = duckdb.sql(vitals48_query).df()
onc_cohort_df = duckdb.sql(onc_cohort).df()

labs48_df

vitals48_df

onc_cohort_df.columns

# Clean & Prepare
labs48_df['charttime'] = pd.to_datetime(labs48_df['charttime'])
labs48_df.dropna(subset=['labs_valuenum'], inplace=True)

# Clean & Prepare
vitals48_df['charttime'] = pd.to_datetime(vitals48_df['charttime'])
vitals48_df.dropna(subset=['vitals_valuenum'], inplace=True)


# --- High Coverage Vitals ---

# 1. Calculate the coverage for each vital label (percentage of unique ICU stays it appears in)
total_unique_stays = vitals48_df['icustay_id'].nunique()
vitals48_coverage = vitals48_df.groupby('vitals_label')['icustay_id'].nunique() / total_unique_stays

# 2. Define your high coverage threshold (e.g., 0.8 for 80%)
coverage_threshold = 0.7

# 3. Filter to get only the labels that meet the high coverage threshold
high_coverage_labels = vitals48_coverage[vitals48_coverage >= coverage_threshold].index.tolist()

print("Labels with High Coverage (>= {}%):".format(coverage_threshold * 100))
print(high_coverage_labels)

# 4. Filter the original DataFrame to include only these high-coverage labels
filtered_vitals_df_high_coverage = vitals48_df[vitals48_df['vitals_label'].isin(high_coverage_labels)]

# 5. Calculate unique ICU stays for each of the high-coverage labels
unique_stays_per_high_coverage_label = filtered_vitals_df_high_coverage.groupby('vitals_label')['icustay_id'].nunique()

# 6. Calculate the percentage of unique ICU stays for each high-coverage label
#    (This is essentially `vitals48_coverage` for the selected labels, but we recalculate
#     it here to fit the structure of the original code's output)
percentage_unique_stays_high_coverage = (unique_stays_per_high_coverage_label / total_unique_stays) * 100

# 7. Combine the counts and percentages for better readability
vitals48_high_coverage_pct_df = pd.DataFrame({
    'Unique ICU Stays Count': unique_stays_per_high_coverage_label,
    'Percentage of Total Unique ICU Stays': percentage_unique_stays_high_coverage
}).sort_values(by='Percentage of Total Unique ICU Stays', ascending=False)

vitals48_high_coverage_pct_df

# --- High Coverage Labs ---

# 1. Calculate the coverage for each vital label (percentage of unique ICU stays it appears in)
total_unique_stays = labs48_df['icustay_id'].nunique()
labs48_coverage = labs48_df.groupby('labs_label')['icustay_id'].nunique() / total_unique_stays

# 2. Define your high coverage threshold (e.g., 0.8 for 80%)
coverage_threshold = 0.7

# 3. Filter to get only the labels that meet the high coverage threshold
high_coverage_labels = labs48_coverage[labs48_coverage >= coverage_threshold].index.tolist()

print("Labels with High Coverage (>= {}%):".format(coverage_threshold * 100))
print(high_coverage_labels)

# 4. Filter the original DataFrame to include only these high-coverage labels
filtered_labs_df_high_coverage = labs48_df[labs48_df['labs_label'].isin(high_coverage_labels)]

# 5. Calculate unique ICU stays for each of the high-coverage labels
unique_stays_per_high_coverage_label = filtered_labs_df_high_coverage.groupby('labs_label')['icustay_id'].nunique()

# 6. Calculate the percentage of unique ICU stays for each high-coverage label
#    (This is essentially `labs48_coverage` for the selected labels, but we recalculate
#     it here to fit the structure of the original code's output)
percentage_unique_stays_high_coverage = (unique_stays_per_high_coverage_label / total_unique_stays) * 100

# 7. Combine the counts and percentages for better readability
labs48_high_coverage_pct_df = pd.DataFrame({
    'Unique ICU Stays Count': unique_stays_per_high_coverage_label,
    'Percentage of Total Unique ICU Stays': percentage_unique_stays_high_coverage
}).sort_values(by='Percentage of Total Unique ICU Stays', ascending=False)

labs48_high_coverage_pct_df

# --- High Coverage Vitals ---

# 1. Calculate the coverage for each vital label (percentage of unique ICU stays it appears in)
total_unique_stays = vitals48_df['icustay_id'].nunique()
vitals48_coverage = vitals48_df.groupby('vitals_label')['icustay_id'].nunique() / total_unique_stays

# 2. Define your high coverage threshold (e.g., 0.8 for 80%)
coverage_threshold = 0.95

# 3. Filter to get only the labels that meet the high coverage threshold
high_coverage_labels = vitals48_coverage[vitals48_coverage >= coverage_threshold].index.tolist()

print("Labels with High Coverage (>= {}%):".format(coverage_threshold * 100))
print(high_coverage_labels)

# 4. Filter the original DataFrame to include only these high-coverage labels
filtered_vitals_df_high_coverage = vitals48_df[vitals48_df['vitals_label'].isin(high_coverage_labels)]

# 5. Calculate unique ICU stays for each of the high-coverage labels
unique_stays_per_high_coverage_label = filtered_vitals_df_high_coverage.groupby('vitals_label')['icustay_id'].nunique()

# 6. Calculate the percentage of unique ICU stays for each high-coverage label
percentage_unique_stays_high_coverage = (unique_stays_per_high_coverage_label / total_unique_stays) * 100

# 7. Combine the counts and percentages for better readability
vitals48_high_coverage_pct_df = pd.DataFrame({
    'Unique ICU Stays Count': unique_stays_per_high_coverage_label,
    'Percentage of Total Unique ICU Stays': percentage_unique_stays_high_coverage
}).sort_values(by='Percentage of Total Unique ICU Stays', ascending=False)

vitals48_high_coverage_pct_df

# Start from your filtered long-form vitals
df = filtered_vitals_df_high_coverage.copy()

# Convert charttime to datetime if not already (redundant if already done, but safe)
df['charttime'] = pd.to_datetime(df['charttime'])

# Feature engineering function per vital per ICU stay
def compute_time_series_features(group):
    # Ensure times and values are numpy arrays for reshape
    times = (group['charttime'] - group['charttime'].min()).dt.total_seconds().values / 3600.0  # in hours
    values = group['vitals_valuenum'].values

    # Defensive check
    if len(values) < 1:
        return pd.Series({'mean': np.nan, 'min': np.nan, 'max': np.nan, 'slope': np.nan})

    # Compute features
    features = {
        'mean': np.mean(values),
        'min': np.min(values),
        'max': np.max(values),
    }

    # Linear regression slope over time
    if len(values) > 1:
        # Use .reshape(-1, 1) on the NumPy array obtained from .values
        model = LinearRegression().fit(times.reshape(-1, 1), values)
        features['slope'] = model.coef_[0]
    else:
        features['slope'] = np.nan

    return pd.Series(features)

# Group by ICU stay and label → apply function
grouped = df.groupby(['icustay_id', 'vitals_label'])
vitals_features_df = grouped.apply(compute_time_series_features, include_groups=False).reset_index()
# Pivot to wide format
vitals_features_wide = vitals_features_df.pivot(index='icustay_id', columns='vitals_label')
vitals_features_wide.columns = [f"{stat.lower()}_{label.lower().replace(' ', '_')}" for stat, label in vitals_features_wide.columns]
vitals_features_wide.reset_index(inplace=True)


vitals_features_wide


# --- High Coverage labs ---

# 1. Calculate the coverage for each vital label (percentage of unique ICU stays it appears in)
total_unique_stays = labs48_df['icustay_id'].nunique()
labs48_coverage = labs48_df.groupby('labs_label')['icustay_id'].nunique() / total_unique_stays

# 2. Define your high coverage threshold (e.g., 0.8 for 80%)
coverage_threshold = 0.95

# 3. Filter to get only the labels that meet the high coverage threshold
high_coverage_labels = labs48_coverage[labs48_coverage >= coverage_threshold].index.tolist()

print("Labels with High Coverage (>= {}%):".format(coverage_threshold * 100))
print(high_coverage_labels)

# 4. Filter the original DataFrame to include only these high-coverage labels
filtered_labs_df_high_coverage = labs48_df[labs48_df['labs_label'].isin(high_coverage_labels)]

# 5. Calculate unique ICU stays for each of the high-coverage labels
unique_stays_per_high_coverage_label = filtered_labs_df_high_coverage.groupby('labs_label')['icustay_id'].nunique()

# 6. Calculate the percentage of unique ICU stays for each high-coverage label
percentage_unique_stays_high_coverage = (unique_stays_per_high_coverage_label / total_unique_stays) * 100

# 7. Combine the counts and percentages for better readability
labs48_high_coverage_pct_df = pd.DataFrame({
    'Unique ICU Stays Count': unique_stays_per_high_coverage_label,
    'Percentage of Total Unique ICU Stays': percentage_unique_stays_high_coverage
}).sort_values(by='Percentage of Total Unique ICU Stays', ascending=False)

labs48_high_coverage_pct_df

# Start from your filtered long-form labs
df = filtered_labs_df_high_coverage.copy()

# Convert charttime to datetime if not already (redundant if already done, but safe)
df['charttime'] = pd.to_datetime(df['charttime'])

# Feature engineering function per vital per ICU stay
def compute_time_series_features(group):
    # Ensure times and values are numpy arrays for reshape
    times = (group['charttime'] - group['charttime'].min()).dt.total_seconds().values / 3600.0  # in hours
    values = group['labs_valuenum'].values

    # Defensive check
    if len(values) < 1:
        return pd.Series({'mean': np.nan, 'min': np.nan, 'max': np.nan, 'slope': np.nan})

    # Compute features
    features = {
        'mean': np.mean(values),
        'min': np.min(values),
        'max': np.max(values),
    }

    # Linear regression slope over time
    if len(values) > 1:
        # Use .reshape(-1, 1) on the NumPy array obtained from .values
        model = LinearRegression().fit(times.reshape(-1, 1), values)
        features['slope'] = model.coef_[0]
    else:
        features['slope'] = np.nan

    return pd.Series(features)

# Group by ICU stay and label → apply function
grouped = df.groupby(['icustay_id', 'labs_label'])
labs_features_df = grouped.apply(compute_time_series_features, include_groups=False).reset_index()
# Pivot to wide format
labs_features_wide = labs_features_df.pivot(index='icustay_id', columns='labs_label')
labs_features_wide.columns = [f"{stat.lower()}_{label.lower().replace(' ', '_')}" for stat, label in labs_features_wide.columns]
labs_features_wide.reset_index(inplace=True)

labs_features_wide


# Merge vitals
onco_feature_df = onc_cohort_df.merge(vitals_features_wide, on='icustay_id', how='left')

# Merge labs
onco_feature_df = onco_feature_df.merge(labs_features_wide, on='icustay_id', how='left')

onco_feature_df


# 1. Calculate the percentage of non-null values for each column
completeness = onco_feature_df.count() / len(onco_feature_df)

# 2. Define a threshold for "most of its rows filled"
completeness_threshold = 0.95

# 3. Filter columns based on the completeness threshold
high_completeness_columns = completeness[completeness >= completeness_threshold].index.tolist()

print(f"Columns with at least {completeness_threshold*100}% of rows filled:")
print(high_completeness_columns)

# 4. Select these columns from the original DataFrame
onco_features_high_completeness = onco_feature_df[high_completeness_columns]

onco_features_high_completeness


# percentage or an absolute number of non-NaN values.
percentage_filled_threshold = 0.95 # Keep rows that are at least 75% filled

# Calculate the minimum number of non-NaN values required based on the percentage
num_columns = onco_features_high_completeness.shape[1]
min_non_nan_for_row = int(num_columns * percentage_filled_threshold)

print(f"Keeping rows with at least {min_non_nan_for_row} non-NaN values (i.e., {percentage_filled_threshold*100}% filled).")

# Drop rows using the calculated threshold
onco_cohort_ML = onco_features_high_completeness.dropna(
    axis=0,        # Operate on rows (default, but good to be explicit)
    thresh=min_non_nan_for_row
).copy()

print("\nDataFrame after dropping rows that are less than {}% filled:".format(percentage_filled_threshold*100))
onco_cohort_ML

# Optional: Impute all missing values (safely: only numeric)
numeric_cols = onco_cohort_ML.select_dtypes(include='number').columns.drop('mortality_30d')

onco_cohort_ML[numeric_cols] = onco_cohort_ML[numeric_cols].fillna(onco_cohort_ML[numeric_cols].median())
onco_cohort_ML

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import pandas as pd

# Load and prepare data
onco_cohort_ML 
X = onco_cohort_ML.drop(columns=['mortality_30d'] + columns_to_drop_from_features, errors='ignore')
y = onco_cohort_ML['mortality_30d']

X = X.select_dtypes(include=[np.number])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Fit XGBoost
model = xgb.XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# SHAP explanation
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# Summary plot
shap.summary_plot(shap_values, X_test, max_display=10)

# Feature importance (mean abs SHAP)
shap_df = pd.DataFrame(shap_values.values, columns=X.columns)
mean_shap = shap_df.abs().mean().sort_values(ascending=False)
print("🔝 Top 10 impactful features:")
print(mean_shap.head(10))




mean_shap.head(10)

top_10_features = mean_shap.head(10).index.tolist()

missing_cols = [col for col in top_10_features if col not in onco_cohort_ML.columns]

if missing_cols:
    print("⚠️ These top SHAP features were not found in the original DataFrame:")
    print(missing_cols)


# Combine identifiers, top SHAP features, and target
all_selected_columns = columns_to_drop_from_features + top_10_features + ['mortality_30d']

# Filter to only columns that exist in the DataFrame
valid_columns = [col for col in all_selected_columns if col in onco_cohort_ML.columns]

# Create the final subset DataFrame
final_feature_subset = onco_cohort_ML[valid_columns].copy()

# Confirm result
print(f"✅ Saved {final_feature_subset.shape[0]} rows × {final_feature_subset.shape[1]} columns")
print("Included columns:", final_feature_subset.columns.tolist())



# Optional: Save the subset
final_feature_subset.to_parquet(
    "/Users/sangeethgeorge/MyProjects/oncoai-patient-outcome-navigator/data/onco_features_cleaned.parquet",
    index=False
)

